In [191]:
from tqdm import tqdm
import pandas as pd
import constants as c
from importlib import reload

reload(c)

<module 'constants' from 'g:\\Shared drives\\Data\\Analysis\\Geography Conversion Tables\\src\\constants.py'>

- Cleaning column and row names

In [193]:
# Loads the geography update, i.e. Postcodes 2016 -> 2021
def load_geo_update_df(geo, Y1, Y2):
    return pd.read_csv(c.geography_updates[geo][Y1 + '-' + Y2])

# With a duplicate, chooses the 2021 Postcode 
def geo_update_sort_drop(df, geo, Y):
    df = df.sort_values(by='RATIO_FROM_TO', ascending=False)
    return df.drop_duplicates(subset= geo + '_NAME_'+ Y, keep='first')

In [194]:
def collapse_transform_table(table, geo1, geo2):
    geo1s, geo2s, households = [], [], []
    geo1_name = c.geo_names[geo1]
    geo2_name = c.geo_names[geo2]
    for col in table.columns:
        subset = table[table[col] > 0][col]
        geo1s += list(subset.index)
        households += list(subset)
        geo2s += list([col])*len(subset)

    geo1_totals = table.sum(axis='columns')
    geo1_totals = geo1_totals.reset_index(name=geo1_name + ' Totals').rename({'index': geo1_name}, axis='columns')

    geo2_totals = table.sum(axis='rows')
    geo2_totals = geo2_totals.reset_index(name=geo2_name + ' Totals').rename({'index': geo2_name}, axis='columns')
    df_new = pd.DataFrame({geo1_name: geo1s, geo2_name: geo2s, 'Rented Households': households})

    df_new = df_new.merge(geo1_totals, on=geo1_name, how='left')
    df_new = df_new.merge(geo2_totals, on=geo2_name, how='left')

    df_new = df_new.sort_values(by=[geo1_name])
    df_new[geo1_name + ' Fraction'] = df_new['Rented Households']/df_new[geo1_name + ' Totals']
    df_new[geo2_name + ' Fraction'] = df_new['Rented Households']/df_new[geo2_name + ' Totals']

    for col in ['Rented Households', geo1_name + ' Totals', geo2_name + ' Totals']:
        df_new[col] = df_new[col].astype(int)

    return(df_new)

    

Update the 2016 geographies to 2021

In [195]:
def update_geos(df, df_geo1, df_geo2, geo1, geo2, Y1, Y2):
    geo1_name, geo2_name = c.geo_names[geo1], c.geo_names[geo2]
    df = df.merge(df_geo1, left_on=geo1_name, right_on=geo1 + '_NAME_' + Y1, how='left')
    df.rename({'RATIO_FROM_TO': geo1 + '_' + Y1 + '_' + Y2 + '_RATIO'}, axis='columns', inplace=True)
    df = df.merge(df_geo2, left_on=geo2_name, right_on=geo2 + '_NAME_' + Y1, how='left')
    df.rename({'RATIO_FROM_TO': geo2 + '_' + Y1 + '_' + Y2 + '_RATIO'}, axis='columns', inplace=True)

    cols = [geo1 + '_NAME_2021', geo2 + '_NAME_2021', 'Rented Households', geo1_name + ' Totals', geo2_name + ' Totals',
       geo1_name + ' Fraction', geo2_name + ' Fraction', geo1 + '_NAME_2016', geo2 + '_NAME_2016', 
       geo1 + '_' + Y1 + '_' + Y2 + '_RATIO', geo2 + '_' + Y1 + '_' + Y2 + '_RATIO'] 
    df = df[cols]

    df.rename({geo1 + '_NAME_' + Y2: geo1_name, geo2 + '_NAME_' + Y2: geo2_name}, axis='columns', inplace=True)
    #Drop missing geos
    df = df[(~df[geo1_name].isna()) & (~df[geo2_name].isna())]

    return(df)


In [ ]:
table_poa_lga = pd.read_excel(c.raw_data_file_lprw, **c.read_options_lprw)

def clean_postcode_name(postcode):
    return(postcode.split(', ')[0].split(' ')[0])

table_poa_lga.rename(clean_postcode_name, axis='index', inplace=True)
table_poa_lga = table_poa_lga[table_poa_lga.columns[1:-1]] #Removes 'LGA' from columns

In [200]:
df_t = load_geo_update_df('LGA', '2016', '2021')
df_lga_2016_to_2021 = geo_update_sort_drop(df_t, 'LGA', '2021')

df_t = load_geo_update_df('POA', '2016', '2021')
df_poa_2016_to_2021 = geo_update_sort_drop(df_t, 'POA', '2021')

df_t = collapse_transform_table(table_poa_lga, 'POA', 'LGA')

df_poa_lga = update_geos(df_t, df_poa_2016_to_2021, df_lga_2016_to_2021, 'POA', 'LGA', '2016', '2021')

In [202]:
df_poa_lga.sort_values(by='Postcode').to_csv(c.processed_data_dir + 'LGA To Postcode 2016 Rented Household Weightings.csv', index=False)